In [1]:
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D,Input,Dropout,Activation
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import CSVLogger,EarlyStopping
import numpy as np
from keras import backend as K
from keras.engine.topology import Layer
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import load_img, img_to_array, array_to_img
import os
import glob
import cv2
from PIL import Image
import matplotlib.pyplot as plt

%matplotlib inline

os.environ["CUDA_VISIBLE_DEVICES"]="0"#specify GPU

image_size_width, image_size_height = (299, 299)

folder = ["CNV_train", "DME_train", "DRUSEN_train", "NORMAL_train"]
x_train = []
y_train = []
for index, name in enumerate(folder):
    dir = "./" + name
    files = glob.glob(dir + "/*.jpeg")
    for i, file in enumerate(files):
        image = Image.open(file)
        image = image.convert("RGB")
        image = image.resize((image_size_width, image_size_height))
        data = np.asarray(image)
        x_train.append(data)
        y_train.append(index)

x_train = np.array(x_train)
y_train = np.array(y_train)
print('x_train.shape', x_train.shape)
print('y_train.shape', y_train.shape)

folder = ["CNV_validation", "DME_validation", "DRUSEN_validation", "NORMAL_validation"]
x_test = []
y_test = []
for index, name in enumerate(folder):
    dir = "./" + name
    files = glob.glob(dir + "/*.jpeg")
    for i, file in enumerate(files):
        image = Image.open(file)
        image = image.convert("RGB")
        image = image.resize((image_size_width, image_size_height))
        data = np.asarray(image)
        x_test.append(data)
        y_test.append(index)

x_test = np.array(x_test)
y_test = np.array(y_test)
print('x_test.shape', x_test.shape)
print('y_test.shape', y_test.shape)

Using TensorFlow backend.


x_train.shape (86647, 299, 299, 3)
y_train.shape (86647,)
x_test.shape (21662, 299, 299, 3)
y_test.shape (21662,)


In [2]:
import keras
from keras import callbacks
from keras.utils import to_categorical

input_shape = (299, 299, 3)
classes = 4
batchsize = 12
epochs=20


base_model=keras.applications.densenet.DenseNet201(input_shape=input_shape,
                                                   weights='imagenet',
                                                   include_top=False)
    
c = base_model.output
c = GlobalAveragePooling2D()(c)
c = keras.layers.Lambda(lambda xx: 5*(xx)/K.sqrt(K.sum(xx**2)))(c) #metric learning
c = Dense(classes, activation='softmax')(c)
model = Model(inputs=base_model.input,outputs=c)

model.summary()

model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=0.001),
                  metrics=['accuracy'])
    
callbacks_list = [
        callbacks.ModelCheckpoint(
        filepath="model.DenseNet201.ep{epoch:02d}.h5",
        save_weights_only=False,
        save_best_only=False)]
    
history = model.fit(x_train, to_categorical(y_train, classes), batch_size=batchsize, epochs=epochs, verbose=1, validation_data=(x_test, to_categorical(y_test, classes)), 
callbacks=callbacks_list)

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 305, 305, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 150, 150, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 150, 150, 64) 256         conv1/conv[0][0]                 
_____________________________________

Instructions for updating:
Use tf.cast instead.
Train on 86647 samples, validate on 21662 samples
Epoch 1/10
86647/86647 [==============================] - 2203s 25ms/step - loss: 0.2254 - acc: 0.9263 - val_loss: 0.1386 - val_acc: 0.9540
Epoch 2/10
86647/86647 [==============================] - 2207s 25ms/step - loss: 0.1352 - acc: 0.9547 - val_loss: 0.1445 - val_acc: 0.9555
Epoch 3/10
86647/86647 [==============================] - 2194s 25ms/step - loss: 0.1113 - acc: 0.9625 - val_loss: 0.1440 - val_acc: 0.9573
Epoch 4/10
86647/86647 [==============================] - 2229s 26ms/step - loss: 0.0977 - acc: 0.9666 - val_loss: 0.1112 - val_acc: 0.9608
Epoch 5/10
86647/86647 [==============================] - 2129s 25ms/step - loss: 0.0843 - acc: 0.9722 - val_loss: 0.0963 - val_acc: 0.9664
Epoch 6/10
86647/86647 [==============================] - 2108s 24ms/step - loss: 0.0769 - acc: 0.9734 - val_loss: 0.0941 - val_acc: 0.9690
Epoch 7/10
86647/86647 [==============================] - 2115